## 326 - Use KDTree and MetPy to Find the Nearest METAR

[Youtube](https://www.youtube.com/watch?v=nMTFb-8e1Nk)

In [1]:
import pandas as pd
import geopandas as gpd
from scipy.spatial import KDTree
from shapely.geometry import Point
from metpy.io import station_info

In [2]:
station_rows = []

for sid, info in station_info.items():
    try:
        lat = getattr(info, 'latitude', None)
        lon = getattr(info, 'longitude', None)
        if lat is not None and lon is not None:
            station_rows.append({'station_id': sid, 'lat': lat, 'lon': lon })
    except Exception as e:
        continue

df = pd.DataFrame(station_rows)

In [3]:
df

,station_id,lat,lon
0,PADK,51.88,-176.65
1,PAKH,56.93,-154.18
2,PAUT,54.15,-165.60
3,PAFM,67.09,-157.85
4,PAKP,68.13,-151.72
...,...,...,...
13786,KQLP,-0.67,36.86
13787,KQKV,13.33,2.60
13788,KQB7,2.57,-72.62
13789,KQB9,0.75,-75.22


In [4]:
coords = df[['lat', 'lon']].values
tree = KDTree(coords)

In [5]:
target_latlon = (36.197473, -95.846057)

In [6]:
distance_deg, idx = tree.query(target_latlon)

In [7]:
nearest_station = df.iloc[idx]

In [8]:
nearest_station

station_id     KTUL
lat            36.2
lon          -95.88
Name: 2502, dtype: object

In [9]:
station_coord = (nearest_station['lat'], nearest_station['lon'])

In [10]:
target_geom = gpd.GeoSeries([Point(target_latlon[1], target_latlon[0])], crs='EPSG:4326')
station_geom = gpd.GeoSeries([Point(station_coord[1], station_coord[0])], crs='EPSG:4326')

In [11]:
target_proj = target_geom.to_crs(epsg=3857)
station_proj = station_geom.to_crs(epsg=3857)

In [12]:
distance_km = target_proj.distance(station_proj).values[0] / 1000

In [13]:
print(f'Target location: {target_latlon}')
print(f"Nearest station: {nearest_station['station_id']}")
print(f'Station location: {station_coord}')
print(f'Distance: {distance_km}')

Target location: (36.197473, -95.846057)
Nearest station: KTUL
Station location: (36.2, -95.88)
Distance: 3.7945632971418854
